# Binary Prediction of Smoker Status using Bio-Signals
[From Kaggle Playground Series](https://www.kaggle.com/competitions/playground-series-s3e24/overview)

In [1]:
import pandas as pd
import torch
import sklearn
import sklearn.model_selection

In [2]:
%pip install torchmetrics
import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 6.6 MB/s eta 0:00:00


In [3]:
%pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')
input_dir = "/content/drive/MyDrive/Data/Playground/"

Mounted at /content/drive


In [5]:
train = pd.read_csv(os.path.join(input_dir, "train.csv"))
evaluate = pd.read_csv(os.path.join(input_dir, "test.csv"))
sample_submission = pd.read_csv(os.path.join(input_dir, "sample_submission.csv"))

In [6]:
print(f"Training data of size = {train.shape}")
print(f"Test data of size = {evaluate.shape}")

Training data of size = (159256, 24)
Test data of size = (106171, 23)


In [7]:
x_columns = train.columns[train.columns != "smoking"]
y_column = train.columns[train.columns == "smoking"]

print("Input data for id = 0")
display(train.iloc[0][x_columns])
print()
print("Target value for id = 0")
display(train.iloc[0][y_column])

Input data for id = 0


id                       0.0
age                     55.0
height(cm)             165.0
weight(kg)              60.0
waist(cm)               81.0
eyesight(left)           0.5
eyesight(right)          0.6
hearing(left)            1.0
hearing(right)           1.0
systolic               135.0
relaxation              87.0
fasting blood sugar     94.0
Cholesterol            172.0
triglyceride           300.0
HDL                     40.0
LDL                     75.0
hemoglobin              16.5
Urine protein            1.0
serum creatinine         1.0
AST                     22.0
ALT                     25.0
Gtp                     27.0
dental caries            0.0
Name: 0, dtype: float64


Target value for id = 0


smoking    1.0
Name: 0, dtype: float64

In [8]:
# Min max normalzie values
x_train = train[x_columns]
x_train_normalized = (x_train-x_train.min())/(x_train.max()-x_train.min())
x_train_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
101756,0.638950,0.384615,0.545455,0.45,0.434211,0.091837,0.091837,0.0,0.0,0.419118,...,0.163588,0.354331,0.038731,0.788820,0.0,0.091837,0.023316,0.008239,0.038114,1.0
40086,0.251710,0.307692,0.545455,0.35,0.381579,0.091837,0.112245,0.0,0.0,0.316176,...,0.133245,0.401575,0.068854,0.559006,0.0,0.051020,0.037565,0.008926,0.095286,0.0
64465,0.404791,0.076923,0.636364,0.35,0.434211,0.051020,0.040816,0.0,0.0,0.301471,...,0.080475,0.322835,0.048413,0.596273,0.0,0.091837,0.018135,0.005149,0.017051,0.0
158450,0.994945,0.615385,0.454545,0.35,0.434211,0.091837,0.142857,0.0,0.0,0.389706,...,0.245383,0.275591,0.041420,0.695652,0.0,0.102041,0.032383,0.011672,0.044132,0.0
36023,0.226197,0.307692,0.454545,0.25,0.280263,0.112245,0.112245,0.0,0.0,0.308824,...,0.055409,0.433071,0.040882,0.496894,0.0,0.071429,0.018135,0.004806,0.022066,0.0


In [9]:
eval_normalized = (evaluate-x_train.min())/(x_train.max()-x_train.min())
eval_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
40764,1.255973,0.076923,0.636364,0.35,0.368421,0.051020,0.081633,0.0,0.0,0.242647,...,0.251979,0.330709,0.053792,0.683230,0.0,0.091837,0.010363,0.006179,0.026078,0.0
79855,1.501435,0.538462,0.545455,0.30,0.315789,0.112245,0.112245,0.0,0.0,0.404412,...,0.258575,0.299213,0.065627,0.639752,0.0,0.102041,0.034974,0.004463,0.048144,0.0
61130,1.383856,0.538462,0.545455,0.40,0.592105,0.051020,0.142857,0.0,0.0,0.382353,...,0.125330,0.291339,0.078537,0.677019,0.0,0.071429,0.049223,0.023687,0.076229,0.0
6811,1.042774,0.538462,0.272727,0.25,0.605263,0.081633,0.081633,0.0,0.0,0.426471,...,0.125330,0.385827,0.093061,0.503106,0.0,0.061224,0.012953,0.003090,0.022066,0.0
84559,1.530972,0.538462,0.727273,0.35,0.473684,0.142857,0.081633,0.0,0.0,0.477941,...,0.365435,0.338583,0.047337,0.645963,0.0,0.102041,0.009067,0.004806,0.015045,0.0


In [10]:
def make_sequential(input_dim, hidden_dim, output_dim, dropout):
    return torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Dropout(p=dropout), torch.nn.Linear(hidden_dim, output_dim))

In [11]:
train_x_tensor = torch.FloatTensor(x_train_normalized.to_numpy())
train_y_tensor = torch.FloatTensor(train[y_column].to_numpy())
eval_x_tensor = torch.FloatTensor(eval_normalized.to_numpy())

print(f"train_x_tensor = {train_x_tensor.shape}",f"train_y_tensor = {train_y_tensor.shape}",f"eval_x_tensor = {eval_x_tensor.shape}",sep="\n")
print(train_x_tensor.min(),train_x_tensor.max())
print(eval_x_tensor.min(),eval_x_tensor.max())

train_x_tensor = torch.Size([159256, 23])
train_y_tensor = torch.Size([159256, 1])
eval_x_tensor = torch.Size([106171, 23])
tensor(0.) tensor(1.)
tensor(-0.0449) tensor(1.6667)


In [24]:
def get_device():
    device = "cpu"
    if torch.cuda.is_available():
      device = "cuda:0"
    return device

class SmokingDataset(torch.utils.data.Dataset):
    def __init__(self,data,target=None):
        self.data = data.to(get_device())

        # Null coalesce the target to an empty of the expected shape.
        target = target if torch.is_tensor(target) else torch.empty((len(data),1))
        self.target = target.to(get_device())

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        return {'data': x, 'target': y}

    def __len__(self):
        return len(self.data)

bsz = 512
tr_x, ts_x, tr_y, ts_y = sklearn.model_selection.train_test_split(train_x_tensor, train_y_tensor)
print(f"tr_x = {tr_x.shape}",f"tr_y = {tr_y.shape}",sep=",\t")
print(f"ts_x = {ts_x.shape}",f"ts_y = {ts_y.shape}",sep=",\t")

train_loader = torch.utils.data.DataLoader(SmokingDataset(tr_x,tr_y), batch_size=bsz)
test_loader = torch.utils.data.DataLoader(SmokingDataset(ts_x,ts_y), batch_size=bsz)

example_batch = next(iter(train_loader))
example_batch["data"].shape, example_batch["target"].shape

tr_x = torch.Size([119442, 23]),	tr_y = torch.Size([119442, 1])
ts_x = torch.Size([39814, 23]),	ts_y = torch.Size([39814, 1])


(torch.Size([512, 23]), torch.Size([512, 1]))

In [26]:
def do_epoch(models,train_loader,test_loader):
  model = models["model"]
  opt = models["opt"]
  loss_fn = models["loss_fn"]
  auroc_fn = models["auroc_fn"]

  for batch in train_loader:
    opt.zero_grad()
    fwd = model(batch["data"])
    loss = loss_fn(fwd,batch["target"])
    loss.backward()
    opt.step()


  with torch.no_grad():
    ts_pred = []
    ts_target = []
    for batch in test_loader:
      ts_pred.append(model(batch["data"]))
      ts_target.append(batch["target"])

  return auroc_fn(torch.cat(ts_pred),torch.cat(ts_target)).cpu().numpy()

def get_config(trial):
  return {
    "lr": trial.suggest_float("lr", 1e-10,1e-1,log=True),
    "dropout": trial.suggest_float("dropout", 0,1),
    "hidden_dim": trial.suggest_int("hidden_dim", 2,512,log=True),
    "epochs": trial.suggest_int("epochs",1,100),
  }

def make_models(config):
  model = make_sequential(len(x_columns),64,1,0)
  model.to(get_device())
  return {
    "loss_fn": torch.nn.BCEWithLogitsLoss(),
    "model": model,
    "opt": torch.optim.Adam(model.parameters()),
    "auroc_fn": torchmetrics.AUROC(task="binary"),
  }

def train_model(trial):
  config = get_config(trial)

  models = make_models(config)

  for i in range(config["epochs"]):
    loss = do_epoch(models,train_loader,test_loader)

    trial.report(loss, i)
    if i % 10 == 0:
      print(i,loss)
    if trial.should_prune():
        raise optuna.TrialPruned()

  return loss

In [17]:
print(f"Num GPUS = {torch.cuda.device_count()}")

Num GPUS = 1


In [27]:
from functools import partial

num_trials = 100

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(train_model, n_trials=num_trials)

print("DONE")
print("CONFIG:", study.best_trial.params)
print("METRICS:", study.best_trial.values)

[I 2023-10-25 18:56:24,558] A new study created in memory with name: no-name-3417a7bc-c1a1-44fc-9cf3-80298cc1c35f


0 0.7943058
10 0.84454393
20 0.8485905
30 0.8511181
40 0.8527726
50 0.853899
60 0.8545434


[I 2023-10-25 18:58:11,286] Trial 0 finished with value: 0.8549607396125793 and parameters: {'lr': 2.581641812336888e-05, 'dropout': 0.6696736113868427, 'hidden_dim': 9, 'epochs': 71}. Best is trial 0 with value: 0.8549607396125793.


70 0.85496074
0 0.79134107
10 0.84451765
20 0.8489367
30 0.8518281


[I 2023-10-25 18:59:04,337] Trial 1 finished with value: 0.8529202938079834 and parameters: {'lr': 8.067035050286012e-09, 'dropout': 0.17697016398362753, 'hidden_dim': 291, 'epochs': 37}. Best is trial 0 with value: 0.8549607396125793.


...

0 0.790746


[I 2023-10-25 19:05:48,195] Trial 97 pruned. 
[I 2023-10-25 19:05:50,053] Trial 98 finished with value: 0.7889877557754517 and parameters: {'lr': 1.1089624420834413e-08, 'dropout': 0.0497203335741003, 'hidden_dim': 13, 'epochs': 1}. Best is trial 0 with value: 0.8549607396125793.


0 0.78898776
0 0.78879035


[I 2023-10-25 19:05:53,415] Trial 99 pruned. 


DONE
CONFIG: {'lr': 2.581641812336888e-05, 'dropout': 0.6696736113868427, 'hidden_dim': 9, 'epochs': 71}
METRICS: [0.8549607396125793]


In [30]:
# Need to remember to make submission predictions with logits
all_loader = torch.utils.data.DataLoader(SmokingDataset(train_x_tensor,train_y_tensor), batch_size=bsz)
eval_loader = torch.utils.data.DataLoader(SmokingDataset(eval_x_tensor), batch_size=bsz)

models = make_models(study.best_trial.params)
for epoch in range(study.best_trial.params["epochs"]):
  loss = do_epoch(models,all_loader,all_loader)
  print(loss)

eval_pred = []
with torch.no_grad():
  for batch in eval_loader:
    eval_pred.append(models["model"](batch["data"]))
eval_prob = torch.nn.functional.sigmoid(torch.cat(eval_pred)).squeeze().cpu()
print(f"eval_prob = {eval_prob.shape} w/ final loss = {loss}")

submission = pd.DataFrame({"id" : evaluate.id, "smoking" : eval_prob}).set_index("id")
display(submission)
submission.to_csv(os.path.join(input_dir, "Submissions", "first.csv"))
print("Made submission!")

0.791667
0.804515
0.81221354
0.8175106
0.82242584
0.826885
0.8307671
0.8337908
0.83618164
0.8382227
0.83982193
0.8413987
0.8427477
0.8441514
0.845156
0.8460775
0.846949
0.8476507
0.8482284
0.84874016
0.8492071
0.84964013
0.85005945
0.8503725
0.85074484
0.85106707
0.8514121
0.85168606
0.85189843
0.85216004
0.85235924
0.8525857
0.852823
0.8529938
0.8531134
0.853216
0.85339797
0.8535762
0.8537128
0.8538149
0.8540041
0.8541497
0.8542941
0.8544691
0.85463715
0.8547028
0.85485375
0.8548567
0.85498375
0.8550711
0.85514414
0.8551978
0.85525537
0.8553271
0.855448
0.85543525
0.8555581
0.85561574
0.855662
0.85570943
0.85579246
0.8558397
0.8559147
0.85595703
0.8559885
0.85605824
0.856125
0.85619485
0.8562278
0.85630155
0.8563665
eval_prob = torch.Size([106171]) w/ final loss = 0.8563665151596069


,smoking
id,
159256,0.475550
159257,0.238837
159258,0.409325
159259,0.031529
159260,0.593042
...,...
265422,0.485163
265423,0.589207
265424,0.328573


Made submission!
